<a href="https://colab.research.google.com/github/tomhyhan/noodles/blob/main/pasta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [20]:
!pwd

/content/noodles


In [6]:
from google.colab import userdata
from google.colab import auth

auth.authenticate_user()
token = userdata.get('token')
username = userdata.get('username')
repo_name = "noodles"

In [8]:
!git clone https://github.com/tomhyhan/{repo_name}.git
%cd {repo_name}
!git pull



/content/noodles


In [16]:
from model.utils import test_colab
test_colab()


Hello Colab! from Noodles ha


In [25]:
!pip install torchinfo
!pip install imagehash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 53.3 MB/s eta 0:00:00


In [33]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold


from config.config_manager import ConfigManager
from collections import Counter
from model.data_model import PastaData
from model.train import trainer, create_model
from model.utils import reset_seed
from model.data import CLASS_ENCODER
from model.viz import class_imblance

In [31]:
config_manager = ConfigManager("./config/config.yml")

SEED = config_manager.config.seed

In [32]:
reset_seed(SEED)

In [ ]:
data = pd.read_csv("./pasta_data.csv")
image_paths, labels = data["img_path"], data["label"]

X, test_data, y, test_label = train_test_split(image_paths.values, labels.values, train_size=0.9, random_state=SEED, shuffle=True, stratify=labels)

In [ ]:
k_fold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
models = ['swin']
for model_name in models:
    for k_id, (train_i, val_i) in enumerate(k_fold.split(X,y)):
        print(len(train_i), len(val_i))
        train_data = X[train_i]
        val_data = y[val_i]

        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=True)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)

        lr = config_manager.config.lr
        num_epochs = config_manager.config.num_epochs
        batch_size = config_manager.config.batch_size
        num_classes = config_manager.config.num_classes
        weight_decay = config_manager.config.weight_decay

        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        model = create_model(model_name)

        loss_history, train_accuracy, val_accuracy, best_accuracy = trainer(
            model,
            train_loader,
            val_loader,
            num_epochs=num_epochs,
            lr=lr,
            batch_size=batch_size,
            k_fold=k_id,
            weight_decay=0.01,
            device=device
        )

        # avg_accuracy = torch.mean(fold_accuracies)
        # std_accuracy = torch.std(fold_accuracies)

        # general plan
        # 1. save best accuracies
        # 2. compare with different models
        # 3. train on full dataset with best performing model
        # 4. make inference


In [41]:
# !git config pull.rebase false
!git pull
# !git config --global user.email "tomhyhan@gmail.com"
# !git config --global user.name username
# !git add .
# !git status
# !git commit -m "Updated Colab notebook"
# !git push https://{token}@github.com/{username}/{repo_name}.git

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 2.23 KiB | 2.23 MiB/s, done.
From https://github.com/tomhyhan/noodles
   0ec2d46..702e7c6  main       -> origin/main
Updating 0ec2d46..702e7c6
Fast-forward
 pasta_classification.ipynb | 257 +++++++++++++++++++++++++++++++++++++++++++++++++++--------------
 1 file changed, 201 insertions(+), 56 deletions(-)
